## Removing Duplicates

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [3]:
# DataFrame method duplicated returns a boolean Series indicating whether each row is a duplicate or not
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [4]:
# drop_duplicates returns a DataFrame where the duplicated array is False
data.drop_duplicates()  # Both of these methods by default consider all the columns

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [5]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [6]:
data.drop_duplicates(['k1'])  # both methods by default keep the first observed value combination

,k1,k2,v1
0,one,1,0
1,two,1,1


In [7]:
# Passing keep='last' will return the last one
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Transforming Data Using a Function or Mapping

In [8]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon','Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [9]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

In [10]:
# The map method on a Series accepts a function or dict-like object containing a mapping
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [11]:
data['animal'] = lowercased.map(meat_to_animal)

In [12]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [13]:
# We could also have passed a function that does all the work
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

Using map is a convenient way to perform element-wise transformations and other data cleaning-related operations

## Replacing Values

In [14]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [16]:
# The -999 values might be sentinel values for missing data. To replace these with NA values that pandas understands,
# we can use replace, producing a new Series (unless you pass inplace = T
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [18]:
# if you want to replace multiple values at once, you instead pass a list and then the substitute value
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [19]:
# to use different replacement for each value, pass a list of substitutes
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [20]:
# the argument passed can also be a dict
data.replace({-999: np.nan, -1000: 1})

0    1.0
1    NaN
2    2.0
3    NaN
4    1.0
5    3.0
dtype: float64

## Renaming Axis Indexes

In [3]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado' , 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [4]:
# like a Series, the axis indexes have a map method
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [5]:
# you can assign to index, modifying the DataFrame in-place
data.index = data.index.map(transform)

In [6]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [7]:
# If you want to create a transformed version of a dataset without modifying the original, a useful method is rename
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [8]:
# Notably, rename can be used in conjunction with a dict-like object providing new values for a subset of the axis label

In [9]:
# rename saves you from the chore of copying the DataFrame manually and assigning to its index and columns attributes
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## Discretization and Binning

In [10]:
# Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [12]:
# Let's divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To do so, we have to use cut, a function in pandas
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [15]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [16]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [17]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

In [18]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [19]:
# You can also pass your own bin names by passing a list or array to the labels option:
group_names = ['Youth', 'YouthAdult', 'MiddleAges', 'Senior']

In [20]:
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YouthAdult', 'Youth', ..., 'YouthAdult', 'Senior', 'MiddleAges', 'MiddleAges', 'YouthAdult']
Length: 12
Categories (4, object): ['Youth' < 'YouthAdult' < 'MiddleAges' < 'Senior']

In [21]:
# If you pass an integer number of bins to cut instead of explicit bin edges, it will compute equal-length bins based on the min and max values
data = np.random.rand(20)  # Uniform distribution (in the range [0, 1))

In [22]:
pd.cut(data, 4, precision=2)

[(0.75, 0.99], (0.51, 0.75], (0.27, 0.51], (0.75, 0.99], (0.025, 0.27], ..., (0.27, 0.51], (0.025, 0.27], (0.75, 0.99], (0.025, 0.27], (0.27, 0.51]]
Length: 20
Categories (4, interval[float64, right]): [(0.025, 0.27] < (0.27, 0.51] < (0.51, 0.75] < (0.75, 0.99]]

In [27]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)
cats

[(-0.0334, 0.651], (-3.362, -0.695], (0.651, 3.123], (-0.0334, 0.651], (0.651, 3.123], ..., (0.651, 3.123], (0.651, 3.123], (-3.362, -0.695], (0.651, 3.123], (-0.695, -0.0334]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.362, -0.695] < (-0.695, -0.0334] < (-0.0334, 0.651] < (0.651, 3.123]]

In [28]:
pd.value_counts(cats)

(-3.362, -0.695]     250
(-0.695, -0.0334]    250
(-0.0334, 0.651]     250
(0.651, 3.123]       250
dtype: int64

In [29]:
pd.value_counts(pd.cut(data, 4))

(-0.119, 1.502]    463
(-1.74, -0.119]    422
(1.502, 3.123]      63
(-3.368, -1.74]     52
dtype: int64

# Detecting and Filtering Outlier

In [30]:
data = pd.DataFrame(np.random.randn(1000, 4))

In [31]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.018039,0.028414,0.027388,0.022620
std,1.010034,0.976656,0.999902,1.029090
min,-2.755827,-3.051313,-3.126386,-2.875836
25%,-0.682926,-0.629087,-0.670911,-0.690499
50%,0.027950,0.034500,0.019979,0.026185
75%,0.729242,0.666165,0.762344,0.742186
max,3.076723,3.138362,2.784265,3.798687


In [33]:
# Suppose you wanted to find values in one of the columns exceeding 3 in absolute value:
col = data[2]
col[np.abs(col) > 3]

194   -3.024400
254   -3.015024
432   -3.126386
Name: 2, dtype: float64

In [34]:
# To select all rows having a value exceeding 3 or -3
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
131,-0.369692,0.233890,-0.092876,3.320618
194,1.415553,0.646839,-3.024400,2.133810
254,0.772341,-0.705112,-3.015024,-0.084373
318,-0.292439,-3.051313,0.366103,0.341748
321,-1.161000,0.079625,0.804535,3.798687
327,0.584894,3.138362,-0.370777,0.675910
432,-1.541956,-1.725491,-3.126386,-0.569191
515,3.076723,1.454420,-0.727397,1.211210


In [35]:
# To cap values outside the interval -3 to 3:
data[np.abs(data) > 3] = np.sign(data) * 3  # The sign function return -1 if x < 0, 0 if x == 0, and 1 if x > 0
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.017962,0.028327,0.027553,0.021501
std,1.009804,0.976064,0.999391,1.025481
min,-2.755827,-3.000000,-3.000000,-2.875836
25%,-0.682926,-0.629087,-0.670911,-0.690499
50%,0.027950,0.034500,0.019979,0.026185
75%,0.729242,0.666165,0.762344,0.742186
max,3.000000,3.000000,2.784265,3.000000


In [36]:
# The statement np.sign(data) produces 1 and -1 values based on whether tha values in data are positive or negative
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,-1.0
1,-1.0,-1.0,1.0,1.0
2,-1.0,1.0,-1.0,1.0
3,1.0,1.0,1.0,-1.0
4,1.0,-1.0,-1.0,1.0


## Permutation and Random Sampling

In [37]:
df = pd.DataFrame(np.arange(20).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [38]:
sampler = np.random.permutation(5)
sampler

array([1, 2, 0, 4, 3])

In [39]:
# sampler array can then be used in iloc-based indexing or the equivalent take function
df.iloc[sampler]

,0,1,2,3
1,4,5,6,7
2,8,9,10,11
0,0,1,2,3
4,16,17,18,19
3,12,13,14,15


In [40]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
2,8,9,10,11
0,0,1,2,3
4,16,17,18,19
3,12,13,14,15


In [41]:
# To select a random subset without replacement, you can use the sample method on Series or DataFrame
df.sample(3)

,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [42]:
# To generate a sample with replacement (to allow repeat choices), pass replace=True to sample
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

1    7
1    7
4    4
0    5
2   -1
3    6
4    4
2   -1
4    4
3    6
dtype: int64

## Computing Indicator/Dummy Variables

In [2]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [3]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [5]:
# In some cases, you may want to add a prefix to the columns in the indicator DataFrame, which can then be merged with the other data.
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [6]:
# If a row in a DataFrame belongs to multiple categories, things are a bit more complicated
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::', header=None, names=mnames)
movies[:10]

/var/folders/7n/5wb06ft57sl0fbrqm6bn40300000gn/T/ipykernel_53377/2071547595.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('datasets/movielens/movies.dat', sep='::', header=None, names=mnames)


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [13]:
# Adding indicator variables for each genre requires a little bit of wrangling.
# First, we extract the list of unique genres in the dataset
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))

In [10]:
genres = pd.unique(all_genres)

In [11]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [14]:
# One way to construct the indicator DataFrame is to start with a DataFrame of all zeros
zero_matrix = np.zeros((len(movies), len(genres)))

In [15]:
zero_matrix.shape

(3883, 18)

In [16]:
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [21]:
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Now iterate through each movie and set entries in each row of dummies to 1
# To do this, we use the dummies.columns to compute the column indices for each genre
gen = movies.genres[0]
gen

"Animation|Children's|Comedy"

In [24]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

In [25]:
# Then we use .iloc to set values based on these indices
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [27]:
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Then, as before, you can combine this with movies
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [31]:
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                              1.0
Genre_Children's                             1.0
Genre_Comedy                                 1.0
Genre_Adventure                              0.0
Genre_Fantasy                                0.0
Genre_Romance                                0.0
Genre_Drama                                  0.0
Genre_Action                                 0.0
Genre_Crime                                  0.0
Genre_Thriller                               0.0
Genre_Horror                                 0.0
Genre_Sci-Fi                                 0.0
Genre_Documentary                            0.0
Genre_War                                    0.0
Genre_Musical                                0.0
Genre_Mystery                                0.0
Genre_Film-Noir                              0.0
Genre_Western       

__For much larger data, this method of constructing indicator variables with multiple membership is not especially speedy. It would
be better to write a lower-level function that writes directly to a NumPy array, and then wrap the result in a DataFrame__

In [32]:
# A useful recipe for statistical applications is to combine get_dummies with a discretization function like cut:
np.random.seed(12345)

In [33]:
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [34]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [35]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0
